In [201]:
import os
import numpy as np
import pandas as pd
import time
import re
import ast
import cssselect
import urllib
import requests
import pprint
import math
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bs4 import BeautifulSoup as bsoup
from lxml import html
from selenium.webdriver import Chrome, ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
pp = pprint.PrettyPrinter(indent=4).pprint
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

In [124]:
webdriver = 'chromedriver_win32/chromedriver.exe'

In [125]:
driver = Chrome(webdriver)#, options=chrome_options)
url = "https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/kernels"
driver.get(url)

In [7]:
actions = ActionChains(driver)
driver.find_element_by_class_name("dataset-header-v2__title").click()
while (driver.find_element_by_class_name("smart-list__message").text != 'No more notebooks to show'):
    actions.key_down(Keys.PAGE_DOWN).perform()

In [419]:
items = driver.find_elements_by_xpath('//*[@id="site-content"]/div[2]/div[2]/div/div/div/div[1]/div[2]/div[2]/div/div/div/div')
for each in items:
    print(each.find_element_by_class_name("kernel-list-item__name").text)

COVID-19 Case Study - Analysis, Viz & Comparisons
CoronaWhy.org - Global Collaboration (join slack)
Browsing research papers with a BM25 search engine
CORD-19: Explore Drugs Being Developed
Topic Modeling: Finding Related Articles
"Learning" Medicine with Word Embeddings ⚕️💊💉
COVID-19 Transmission and incubation
Visualization of Virus Origin & Genetic Evolution
Air Temperature and COVID-19
Anserini+BERT-SQuAD for context corpus search
COVID-19 Four different epidemic situations
covid-19-incubation-time
A CORD19 Research Paper Search Engine
Age Dependent Incubation Period
Asking questions with a BM25/BERT
Corona Virus Data Visualization
3D World Heat Map of Cumulative Confirmed Case
Exploratory Data Analysis covid-19 from eCDC
CORD-19 Metadata Enrich [2/x]: Altmetric API
Convert COVID-19 papers to Entity Vectors
CORD-19 Topic Modelling
NLP Text Mining - Disease behavior
COVID-2019-ID Paper version
CORD-19 Analysis with Sentence Embeddings
Utilizing BioBERT for K-means Topic Clustering
A

CORD-19: interactive word2vec paragraph search
COVID-19 (Task 01)
COVID-19 Word Embedding Approach
[CORD-19] Embeddings 📙 from abstracts with SpaCy
symptoms word cloud
Detecting and Visualizing covid-19
[CORD-19] Embeddings 📙 from abstracts with SpaCy
NG-EDA-COVID-19-v2
COVID-19 World Statistics
Extract Tika cTAKES Features
Covid-19 Analysis
Extractive text summarization
COVID-19: Extracting the Hidden Topics With Gensim
COVID-19 - Interactive Cluster Graph
Papers on Virus Genetics
Enigma Covid-19
BM25 Search + Query Similarity Ranking
COVID parameter study - India & others
naive LSTM model test for covid19
Search functions for cord19-challenge
Browsing research papers with a BM25 search engine
COVID-19 Dataset : Gaining actionable insights 📊
What Are The Treatments?
Tracking the spread of the Coronavirus
Clustering Papers using K-Means and t-SNE
Find similar COVID-19 Research Articles
COVID-19 Classify articles-Doc2vec PCA
Build CSV of Body Text
betweenness_centrality_of_the_bibliogra

In [413]:
print(items[23].get_attribute('innerHTML'))

<div class="kernel-list-item__meta"><div class="kernel-list-item__votes"><div class="vote-button__container "><div class="vote-button vote-button--enabled"><div class="vote-button__button vote-button__button--up vb-upvote"><span class="fa fa-caret-up"></span></div><div class="vote-button__button vote-button__button--up vote-button__vote-count-container"><span class="vote-button__vote-count">57</span></div><div class="vote-button__button-placeholder"></div></div></div></div><div class="kernel-list-item__image-wrapper"><span class="tooltip-container" data-tooltip="David Mezzetti"><a class="avatar" href="/nofoosports" style="width: 36px; position: relative; z-index: 2; pointer-events: auto;"><img class="avatar__thumbnail" src="https://storage.googleapis.com/kaggle-avatars/thumbnails/555186-kg.png" alt="David Mezzetti" width="36" height="36" style="border-radius: 3.6px;"><img class="avatar__tier" src="/static/images/avatier/avatier-contributor@2x.png" alt="contributor tier" width="36" styl

# Features
- votes
- user
- tier
- notebook_name
- time_published
- relative_time_published
- tags
- outputs
- language
- num_comments
- link

In [444]:
metadata = {'votes': [], 'user_link': [], 'tier': [], 'notebook_name': [], 
            'num_visualizations': [], 'num_datafiles': [], 'time_published': [], 
            'relative_time_published': [], 'tags': [], 'language': [], 
            'num_comments': [], 'notebook_link': []}

In [445]:
tidx = 6 #test index of items
print(items[tidx].find_element_by_class_name("vote-button__vote-count").text)
print(items[tidx].find_element_by_class_name('avatar').get_attribute("href"))
print(items[tidx].find_element_by_class_name('avatar__tier').get_attribute('alt').split(' ')[0])
print(items[tidx].find_element_by_class_name('kernel-list-item__name').text)
print(items[tidx].find_element_by_class_name('kernel-list-item__details').\
                  find_element_by_xpath('.//*[@title]').get_attribute('title'))
print(items[tidx].find_element_by_class_name('kernel-list-item__details').\
                  find_element_by_xpath('.//*[@title]').text)
print([each.text for each in items[tidx].find_elements_by_class_name('Tag_TextAnchor-sc-hezo17') if each.text != ''])
tooltip = items[tidx].find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[1]').get_attribute('data-tooltip')[:-1].split(' ')
                                                                     # remove the period
if ('visualizations' in tooltip) or ('visualization' in tooltip):
    for i, v in enumerate(tooltip):
        if ((v == 'visualizations' or v == 'visualization') and (tooltip[i-1].isdigit())):
            print('vis', tooltip[i-1])
        elif ((v == 'visualizations' or v == 'visualization') and (not tooltip[i-1].isdigit())):
            print('vis', 0)
else:
    print('vis', 0)
if ('data' in tooltip):
    for i, v in enumerate(tooltip):            
        if ((v == 'data') and (tooltip[i-1].isdigit())):
            print('data', tooltip[i-1])
        elif ((v == 'data') and (not tooltip[i-1].isdigit())):
            print('data', 0)
else:
    print('data', 0)
#always (visualizations then data files) when both present

print(items[tidx].find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[2]').get_attribute("data-tooltip").split(' ')[-1:][0])
print(re.sub('\D', '', items[tidx].find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[3]').get_attribute("data-tooltip")))
print(items[tidx].find_element_by_xpath('.//a').get_attribute('href'))

6
https://www.kaggle.com/ajrwhite
contributor
COVID-19 Transmission and incubation
Sat Mar 28 2020 13:50:24 GMT-0400 (Eastern Daylight Time)
4h ago
['covid19']
vis 0
data 1
Python
4
https://www.kaggle.com/ajrwhite/covid-19-transmission-and-incubation


In [298]:
items[0].get_attribute('class')

'block-link block-link--bordered'

In [341]:
print(html.tostring(items[0]))

b'<div class="block-link block-link--bordered"><a class="block-link__anchor" href="/dgunning/browsing-research-papers-with-a-bm25-search-engine"></a><span><div class="kernel-list-item"><div class="kernel-list-item__meta"><div class="kernel-list-item__votes"><div class="vote-button__container "><div class="vote-button vote-button--enabled"><div class="vote-button__button vote-button__button--up vb-upvote"><span class="fa fa-caret-up"></span></div><div class="vote-button__button vote-button__button--up vote-button__vote-count-container"><span class="vote-button__vote-count">287</span></div><div class="vote-button__button-placeholder"></div></div></div></div><div class="kernel-list-item__image-wrapper"><span class="tooltip-container" data-tooltip="DwightGunning"><a class="avatar" href="/dgunning" style="width: 36px; position: relative; z-index: 2; pointer-events: auto;"><img class="avatar__thumbnail" src="https://storage.googleapis.com/kaggle-avatars/thumbnails/396930-gp.jpg" alt="DwightG

In [491]:
html.tostring(tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/span[1]")[13])

b'<span><span><div class="ToolTip_ToolTipContainer-sc-f0vhmk eHUYTV"><span class="fa fa-users kernel-list-item__collaboration-icon" data-tip="true" data-for="tooltip_1" currentitem="false"></span><div class="__react_component_tooltip place-top type-dark " id="tooltip_1" data-id="tooltip"><div class="ToolTip_ToolTipView-sc-1ci7zcv johoye"><span>This Notebook has collaborators.</span></div></div></div></span></span>'

In [489]:
for i, each in enumerate(tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/span[1]")):
    print (each.attrib['title'], i)

Mon Mar 30 2020 09:00:51 GMT-0400 (Eastern Daylight Time) 0
Mon Mar 30 2020 10:58:23 GMT-0400 (Eastern Daylight Time) 1
Mon Mar 30 2020 10:12:42 GMT-0400 (Eastern Daylight Time) 2
Mon Mar 30 2020 16:31:53 GMT-0400 (Eastern Daylight Time) 3
Mon Mar 30 2020 06:35:53 GMT-0400 (Eastern Daylight Time) 4
Sun Mar 29 2020 23:32:57 GMT-0400 (Eastern Daylight Time) 5
Mon Mar 30 2020 12:10:39 GMT-0400 (Eastern Daylight Time) 6
Mon Mar 30 2020 14:56:48 GMT-0400 (Eastern Daylight Time) 7
Mon Mar 30 2020 16:05:26 GMT-0400 (Eastern Daylight Time) 8
Mon Mar 30 2020 08:35:19 GMT-0400 (Eastern Daylight Time) 9
Mon Mar 30 2020 14:15:38 GMT-0400 (Eastern Daylight Time) 10
Mon Mar 30 2020 14:14:08 GMT-0400 (Eastern Daylight Time) 11
Mon Mar 30 2020 15:02:03 GMT-0400 (Eastern Daylight Time) 12


KeyError: 'title'

In [78]:
[each.text for each in items[0].find_elements_by_class_name('Tag_TextAnchor-sc-hezo17') if each.text != '']


AttributeError: 'HtmlElement' object has no attribute 'find_elements_by_class_name'

In [80]:
tree.xpath(".//span/div/div[2]/div/div/span[2]/*/*[self::a]")

[<Element a at 0x27b59d75e08>,
 <Element a at 0x27b59d77818>,
 <Element a at 0x27b59d77688>,
 <Element a at 0x27b59d77908>,
 <Element a at 0x27b59d77a48>,
 <Element a at 0x27b59d77ae8>,
 <Element a at 0x27b59d77b38>,
 <Element a at 0x27b59d77b88>,
 <Element a at 0x27b59d77bd8>,
 <Element a at 0x27b59d77c28>,
 <Element a at 0x27b59d77c78>,
 <Element a at 0x27b59d77cc8>,
 <Element a at 0x27b59d77d18>,
 <Element a at 0x27b59d77d68>,
 <Element a at 0x27b59d77db8>,
 <Element a at 0x27b59d77e08>,
 <Element a at 0x27b59d77e58>,
 <Element a at 0x27b59d77ea8>,
 <Element a at 0x27b59d77ef8>,
 <Element a at 0x27b59d77f48>,
 <Element a at 0x27b59d77f98>,
 <Element a at 0x27b59d6f048>,
 <Element a at 0x27b59d6f098>,
 <Element a at 0x27b59d6f0e8>,
 <Element a at 0x27b59d6f138>,
 <Element a at 0x27b59d6f188>,
 <Element a at 0x27b59d6f1d8>,
 <Element a at 0x27b59d6f228>,
 <Element a at 0x27b59d6f278>,
 <Element a at 0x27b59d6f2c8>,
 <Element a at 0x27b59d6f318>,
 <Element a at 0x27b59d6f368>,
 <Elemen

In [ ]:
//*[@id="site-content"]/div[2]/div[2]/div/div/div/div[1]/div[2]/div[2]/div/div/div[1]/div/span/div/div[2]/div[2]/span[1]

In [130]:
[[each.text for each in block.xpath(".//span/div/div[2]/div/div/span[2]/*/*[self::a]")] for block in tree.xpath("//*[@class='block-link block-link--bordered']")]

[['epidemiology', 'health', 'data visualization', 'starter code', 'covid19'],
 ['beginner', 'eda', 'starter code'],
 [],
 [],
 [],
 [],
 ['nlp', 'classification', 'clustering'],
 ['eda', 'nlp', 'covid19'],
 [],
 ['nlp', 'data visualization', 'classification', 'starter code'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['health', 'beginner', 'nlp', 'data visualization', 'starter code'],
 [],
 [],
 ['data visualization'],
 [],
 [],
 [],
 [],
 ['tutorial', 'data visualization', 'starter code'],
 ['epidemiology', 'text mining', 'recommendation', 'spaCy', 'covid19'],
 [],
 ['data journalism'],
 ['beginner', 'eda', 'data cleaning'],
 [],
 [],
 ['covid19'],
 ['medicine', 'eda', 'nlp', 'text data', 'covid19'],
 ['eda', 'nlp', 'feature engineering', 'starter code', 'covid19'],
 [],
 ['covid19'],
 ['covid19'],
 ['covid19'],
 ['covid19'],
 [],
 ['covid19'],
 ['covid19'],
 ['covid19'],
 ['data cleaning', 'data management', 'covid19'],
 [],
 [],
 ['covid19'],
 [],
 [],
 ['research', 'nlp', 'covid19'],
 [],
 []

In [383]:
driver.find_element_by_class_name('avatar').get_attribute("href")

'https://www.kaggle.com/dgunning'

# LXML rewrite, i think over 100x speed increase lol (maybe not, but still pretty fast)

In [122]:
def parsevis(tooltip):
    vis = 0
    if ('visualizations' in tooltip) or ('visualization' in tooltip):
        for i, v in enumerate(tooltip):
            if ((v == 'visualizations' or v == 'visualization') and (tooltip[i-1].isdigit())):
                vis = int(tooltip[i-1])
            elif ((v == 'visualizations' or v == 'visualization') and (not tooltip[i-1].isdigit())):
                vis = 0
    else:
        vis = 0
    return vis

def parsedata(tooltip):
    data = 0
    if ('data' in tooltip):
        for i, v in enumerate(tooltip):            
            if ((v == 'data') and (tooltip[i-1].isdigit())):
                data = int(tooltip[i-1])
            elif ((v == 'data') and (not tooltip[i-1].isdigit())):
                data = 0
    else:
        data = 0
    return data

In [131]:
metadata = {'votes': [], 'user_link': [], 'tier': [], 'notebook_name': [], 
            'num_visualizations': [], 'num_datafiles': [], 'time_published': [], 
            'relative_time_published': [], 'tags': [], 'language': [], 
            'num_comments': [], 'notebook_link': []}
metadata['votes'] = [each.text for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*/*/*[@class='vote-button__vote-count']")]
metadata['user_link'] = ['https://kaggle.com'+ each.attrib['href'] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@class='avatar']")]
metadata['tier'] = [each.attrib['alt'].split(' ')[0] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*/*[@class='avatar__tier']")]
metadata['notebook_name'] = [each.text for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@class ='kernel-list-item__name false']")]              
metadata['time_published'] = [each.attrib['title'] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@title and self::span]")]
metadata['relative_time_published'] = [each.text for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@title and self::span]")]
metadata['tags'] = [[each.text for each in block.xpath(".//span/div/div[2]/div/div/span[2]/*/*[self::a]")] for block in tree.xpath("//*[@class='block-link block-link--bordered']")]
metadata['language'] = [each.attrib['data-tooltip'].split(' ')[-1:][0] for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[2]")]
metadata['num_comments'] = [re.sub('\D','',each.attrib['data-tooltip']) for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[3]")]
metadata['notebook_link'] = ['https://kaggle.com' + each.attrib['href'] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*[self::a]")]
metadata['num_visualizations'] = np.array([parsevis(each.attrib['data-tooltip'][:-1].split(' ')) for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[1]")])
metadata['num_datafiles'] = np.array([parsedata(each.attrib['data-tooltip'][:-1].split(' ')) for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[1]")])

In [386]:
metadata['user_link']

['/dgunning', '/mobassir', '/dirktheeng', '/mlconsult', '/phyothuhtet', '/nofoosports', '/yuanso', '/marcusgawronsky', '/tarunkr', '/dannellyz', '/jiripodivin', '/psrajput', '/gatunnopvp', '/maria17', '/duttadebadri', '/arturkiulian', '/raymondjia', '/maonanwang', '/ajrwhite', '/ritam3144', '/maksimeren', '/mlconsult', '/chuntolee', '/darensin01', '/danielwolffram', '/mikehatchi', '/narasimha1997', '/jpmiller', '/xhlulu', '/nofoosports', '/nofoosports', '/nofoosports', '/ajrwhite', '/nofoosports', '/isaacmg', '/nofoosports', '/nofoosports', '/nofoosports', '/nofoosports', '/daxman', '/tarunpaparaju', '/ahmedewida', '/nofoosports', '/izzatalsmadi', '/uplytics', '/ravitatakc', '/lalitapatel', '/gpreda', '/shahules', '/something4kag', '/nofoosports', '/dannellyz', '/latong', '/sixteenpython', '/cstefanache', '/ratan123', '/saga21', '/group16', '/d4v1d3', '/chaker1', '/ztsincom', '/tanulsingh077', '/paultimothymooney', '/dangelov', '/fmitchell259', '/anuraagsaini', '/ivanegapratama', '/ham

In [47]:
tree = html.fromstring(driver.page_source)
items = tree.xpath("//*[@class='block-link block-link--bordered']")

In [308]:
items = driver.find_elements_by_xpath('//*[@id="site-content"]/div[2]/div[2]/div/div/div/div[1]/div[2]/div[2]/div/div/div/div')


# Original Selenium Parsing :thumbsdown:

In [446]:
metadata = {'votes': [], 'user_link': [], 'tier': [], 'notebook_name': [], 
            'num_visualizations': [], 'num_datafiles': [], 'time_published': [], 
            'relative_time_published': [], 'tags': [], 'language': [], 
            'num_comments': [], 'notebook_link': []}
tree = html.fromstring(driver.page_source)
items = tree.xpath("//*[@class='block-link block-link--bordered']")
for i, each in enumerate(items):
    if (i%50==0):
        print(i)
    metadata['votes'].append(int(each.find_element_by_class_name("vote-button__vote-count").text))
    metadata['user_link'].append(each.find_element_by_class_name('avatar').get_attribute("href"))
    metadata['tier'].append(each.find_element_by_class_name('avatar__tier').get_attribute('alt').split(' ')[0])
    metadata['notebook_name'].append(each.find_element_by_class_name('kernel-list-item__name').text)
    tooltip = each.find_element_by_class_name('kernel-list-item__info-blocks')\
              .find_element_by_xpath('.//span[1]').get_attribute('data-tooltip')[:-1].split(' ')
    if ('visualizations' in tooltip) or ('visualization' in tooltip):
        for i, v in enumerate(tooltip):
            if ((v == 'visualizations' or v == 'visualization') and (tooltip[i-1].isdigit())):
                metadata['num_visualizations'].append(int(tooltip[i-1]))
            elif ((v == 'visualizations' or v == 'visualization') and (not tooltip[i-1].isdigit())):
                metadata['num_visualizations'].append(0)
    else:
        metadata['num_visualizations'].append(0)
    if ('data' in tooltip):
        for i, v in enumerate(tooltip):            
            if ((v == 'data') and (tooltip[i-1].isdigit())):
                metadata['num_datafiles'].append(int(tooltip[i-1]))
            elif ((v == 'data') and (not tooltip[i-1].isdigit())):
                metadata['num_datafiles'].append(0)
    else:
        metadata['num_datafiles'].append(0)
    metadata['time_published'].append(each.find_element_by_class_name('kernel-list-item__details').\
                                          find_element_by_xpath('.//*[@title]').get_attribute('title'))
    metadata['relative_time_published'].append(each.find_element_by_class_name('kernel-list-item__details').\
                                              find_element_by_xpath('.//*[@title]').text)
    metadata['tags'].append([each.text for each in\
                             each.find_elements_by_class_name('Tag_TextAnchor-sc-hezo17')\
                             if each.text != ''])
    metadata['language'].append(each.find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[2]').get_attribute("data-tooltip").split(' ')[-1:][0])
    metadata['num_comments'].append(re.sub('\D', '', each.find_element_by_class_name('kernel-list-item__info-blocks')\
                  .find_element_by_xpath('.//span[3]').get_attribute("data-tooltip")))
    metadata['notebook_link'].append(each.find_element_by_xpath('.//a').get_attribute('href'))

0
50
100
150
200
250
300
350
400


# Full code

In [136]:
def parsevis(tooltip):
    vis = 0
    if ('visualizations' in tooltip) or ('visualization' in tooltip):
        for i, v in enumerate(tooltip):
            if ((v == 'visualizations' or v == 'visualization') and (tooltip[i-1].isdigit())):
                vis = int(tooltip[i-1])
            elif ((v == 'visualizations' or v == 'visualization') and (not tooltip[i-1].isdigit())):
                vis = 0
    else:
        vis = 0
    return vis

def parsedata(tooltip):
    data = 0
    if ('data' in tooltip):
        for i, v in enumerate(tooltip):            
            if ((v == 'data') and (tooltip[i-1].isdigit())):
                data = int(tooltip[i-1])
            elif ((v == 'data') and (not tooltip[i-1].isdigit())):
                data = 0
    else:
        data = 0
    return data

webdriver = 'chromedriver_win32/chromedriver.exe'

driver = Chrome(webdriver)#, options=chrome_options)
url = "https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/kernels"
driver.get(url)
print ('started webdriver, loading elements')

actions = ActionChains(driver)
driver.find_element_by_class_name("dataset-header-v2__title").click()
while (driver.find_element_by_class_name("smart-list__message").text != 'No more notebooks to show'):
    actions.key_down(Keys.PAGE_DOWN).perform()
print('loaded elements, scraping')
    
tree = html.fromstring(driver.page_source)
metadata = {'votes': [], 'user_link': [], 'tier': [], 'notebook_name': [], 
            'num_visualizations': [], 'num_datafiles': [], 'time_published': [], 
            'relative_time_published': [], 'tags': [], 'language': [], 
            'num_comments': [], 'notebook_link': []}
metadata['votes'] = [each.text for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*/*/*[@class='vote-button__vote-count']")]
metadata['user_link'] = ['https://kaggle.com'+ each.attrib['href'] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@class='avatar']")]
metadata['tier'] = [each.attrib['alt'].split(' ')[0] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*/*[@class='avatar__tier']")]
metadata['notebook_name'] = [each.text for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@class ='kernel-list-item__name false']")]              
metadata['time_published'] = [each.attrib['title'] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@title and self::span]")]
metadata['relative_time_published'] = [each.text for each in tree.xpath("//*[@class='block-link block-link--bordered']/*/*/*/*/*/*[@title and self::span]")]
metadata['tags'] = [[each.text for each in block.xpath(".//span/div/div[2]/div/div/span[2]/*/*[self::a]")] for block in tree.xpath("//*[@class='block-link block-link--bordered']")]
metadata['language'] = [each.attrib['data-tooltip'].split(' ')[-1:][0] for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[2]")]
metadata['num_comments'] = [re.sub('\D','',each.attrib['data-tooltip']) for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[3]")]
metadata['notebook_link'] = ['https://kaggle.com' + each.attrib['href'] for each in tree.xpath("//*[@class='block-link block-link--bordered']/*[self::a]")]
metadata['num_visualizations'] = np.array([parsevis(each.attrib['data-tooltip'][:-1].split(' ')) for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[1]")])
metadata['num_datafiles'] = np.array([parsedata(each.attrib['data-tooltip'][:-1].split(' ')) for each in tree.xpath("//*[@class='block-link block-link--bordered']/span/div/div[2]/div[2]/span[1]")])
print('done scraping, writing csv')

meta_df = pd.DataFrame.from_dict(metadata)
meta_df.to_csv("covid-kernels.csv", index = False)
print('done writing csv')

started webdriver, loading elements
loaded elements, scraping
done scraping, writing csv
done writing csv


In [447]:
metadata

{'votes': [111,
  87,
  272,
  36,
  181,
  3,
  6,
  1,
  3,
  13,
  10,
  1,
  2,
  1,
  3,
  4,
  0,
  1,
  0,
  0,
  0,
  44,
  0,
  57,
  4,
  1,
  404,
  12,
  283,
  0,
  265,
  199,
  0,
  1,
  2,
  97,
  95,
  94,
  1,
  18,
  15,
  63,
  1,
  25,
  48,
  55,
  0,
  0,
  52,
  28,
  46,
  43,
  0,
  0,
  24,
  0,
  0,
  3,
  0,
  13,
  10,
  0,
  17,
  27,
  6,
  16,
  1,
  2,
  6,
  15,
  23,
  0,
  6,
  9,
  19,
  19,
  0,
  0,
  17,
  2,
  17,
  3,
  1,
  0,
  5,
  15,
  5,
  5,
  15,
  6,
  14,
  14,
  2,
  0,
  1,
  3,
  13,
  3,
  12,
  12,
  12,
  1,
  11,
  10,
  10,
  10,
  9,
  0,
  5,
  1,
  8,
  8,
  8,
  9,
  8,
  8,
  0,
  7,
  7,
  7,
  7,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  7,
  6,
  3,
  5,
  5,
  6,
  5,
  5,
  6,
  5,
  5,
  4,
  4,
  4,
  4,
  2,
  4,
  1,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  3,
  0,
  1,
  0,
  1,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  1,
  1

In [132]:
meta_df = pd.DataFrame.from_dict(metadata)

In [133]:
meta_df

,votes,user_link,tier,notebook_name,num_visualizations,num_datafiles,time_published,relative_time_published,tags,language,num_comments,notebook_link
0,165,https://kaggle.com/tarunkr,novice,"COVID-19 Case Study - Analysis, Viz & Comparisons",53,0,Mon Mar 30 2020 16:05:26 GMT-0400 (Eastern Day...,2h ago,"[epidemiology, health, data visualization, sta...",Python,68,https://kaggle.com/tarunkr/covid-19-case-study...
1,287,https://kaggle.com/dgunning,novice,Browsing research papers with a BM25 search en...,0,0,Mon Mar 30 2020 09:00:51 GMT-0400 (Eastern Day...,9h ago,"[beginner, eda, starter code]",Python,32,https://kaggle.com/dgunning/browsing-research-...
2,107,https://kaggle.com/mobassir,master,😷Mining COVID-19 scientific papers🦠,7,5,Mon Mar 30 2020 10:58:23 GMT-0400 (Eastern Day...,7h ago,[],Python,25,https://kaggle.com/mobassir/mining-covid-19-sc...
3,25,https://kaggle.com/dirktheeng,novice,Anserini+BERT-SQuAD for context corpus search,0,0,Mon Mar 30 2020 17:00:31 GMT-0400 (Eastern Day...,1h ago,[],Python,17,https://kaggle.com/dirktheeng/anserini-bert-sq...
4,16,https://kaggle.com/duttadebadri,expert,COVID-19 Analysing Growth Factor & Inflection,2,0,Mon Mar 30 2020 16:40:10 GMT-0400 (Eastern Day...,1h ago,[],Python,6,https://kaggle.com/duttadebadri/covid-19-analy...
...,...,...,...,...,...,...,...,...,...,...,...,...
488,0,https://kaggle.com/srulikbd,novice,SciBERT+sentence similarity,0,0,Thu Mar 19 2020 04:48:42 GMT-0400 (Eastern Day...,12d ago,[],Python,0,https://kaggle.com/srulikbd/scibert-sentence-s...
489,0,https://kaggle.com/sudhendu,contributor,CORD-19 Search articles with Doc2Vec,0,0,Wed Mar 18 2020 13:34:21 GMT-0400 (Eastern Day...,12d ago,[],Python,0,https://kaggle.com/sudhendu/cord-19-search-art...
490,0,https://kaggle.com/mobasshir,contributor,Creating a Good Analytics Report,0,0,Mon Mar 30 2020 08:59:56 GMT-0400 (Eastern Day...,9h ago,[],Python,0,https://kaggle.com/mobasshir/creating-a-good-a...
491,0,https://kaggle.com/skblaz,novice,CORVID interactive,0,0,Thu Mar 19 2020 09:44:08 GMT-0400 (Eastern Day...,11d ago,[],Python,0,https://kaggle.com/skblaz/corvid-interactive


In [4]:
meta_df.to_csv("covid-kernels.csv", index = False)

NameError: name 'meta_df' is not defined

In [5]:
meta_df = pd.read_csv("covid-kernels.csv")

# Notebook Scraping

In [4]:
driver.close()

In [55]:
driver = Chrome(webdriver)#, options=chrome_options)
url = meta_df['notebook_link'][0]
driver.get(url)
driver.implicitly_wait(10)

In [257]:
driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="rendered-kernel-content"]'))

In [7]:
h1 = [each.text for each in driver.find_elements_by_tag_name('h1')]
h2 = [each.text for each in driver.find_elements_by_tag_name('h2')]
h3 = [each.text for each in driver.find_elements_by_tag_name('h3')]
h4 = [each.text for each in driver.find_elements_by_tag_name('h4')]
h5 = [each.text for each in driver.find_elements_by_tag_name('h5')]
h6 = [each.text for each in driver.find_elements_by_tag_name('h6')]

In [24]:
dictionary = ['trend', 'map', 'curve', 'visualization', 'visualisation',
              'prediction', 'tabulation', 'table', 'histogram', 'bar', 'figure']

In [37]:
[each.text for each in driver.find_element_by_id("Content-:").find_element_by_xpath("..").find_elements_by_tag_name('li')]

['Intorduction to COVID-19',
 'Downloding and Installing Prerequisite',
 'Imports and Datasets',
 'Preprocessing',
 'Defining Functions',
 'General Analysis of Data',
 'Correlation Analysis',
 'Visualization on Map\nGlobal Confirmed Cases Heat Map\nGlobal Deaths Heat Map',
 'Global Confirmed Cases Heat Map',
 'Global Deaths Heat Map',
 'COVID-19 Spread Analysis\nSpread Comparison with India New\nCOVID-19 Spread Comparison of in different continents New',
 'Spread Comparison with India New',
 'COVID-19 Spread Comparison of in different continents New',
 'COVID-19 Case (Confirmed, Deaths, Recovered and Active) Percent Split in Different countries (Pie Viz)',
 'Global Prediction\nTabulation of prediction and actual figure after 24th March (Global) New',
 'Tabulation of prediction and actual figure after 24th March (Global) New',
 'COVID-19 Mortality Rate Variation Over Period of Time',
 'Comparision of Mortality and Recovery Rate',
 'Cumulative Confirmed Cases and Cumulative Recovery Vs C

# we should probably just scrape all the imports then graph them lmfao

A notebook with an unfamiliar import can lead to you getting more information about different data science technniques
A notebook with low level imports can signal that this person is making their own model, etc.
A notebook with cosmetic imports can help you add to your toolkit


In [11]:
len(driver.find_element_by_xpath('//*[@id="notebook"]').text)

60436

In [258]:
#load lxml tree
tree = html.fromstring(driver.page_source)

In [227]:
len(str([each.text for each in tree.body.xpath('//*[self::p or self::strong or self::h1 or self::h2 or self::h3 or self::h4]')]))

12981

In [143]:
#Get views
tree.xpath('//*[@id="kernel-header-wrapper"]/div[1]/span[1]/span[2]/span/span[2]/span')[0].text

'16,735'

In [173]:
str([each.text for each in tree.xpath('//*[@id="rendered-kernel-content"]')])#[self::h1 or self::h2 or self::h3 or self::p]')])

'[None]'

In [272]:
    code = [each.text for each in driver.find_elements_by_class_name("input_area") if 'import' in each.text]
    modules = []
    for block in code:
        for each in block.split('\n'):
            tokens = each.split(' ')
            if tokens[0] == 'import':
                for package in re.sub(' ', '', each.split(' as ')[0].split('import')[1]).split(','):
                    modules.append(package)
            elif tokens[0] == 'from':
                submodules = each.split('import')[1]
                for sub in re.sub(' ', '', submodules).split(','):
                    modules.append(tokens[1]+'.'+sub)


In [276]:
modules

['ipywidgets.Image',
 'cord.ResearchPapers',
 'multiprocessing',
 'concurrent.futures.ThreadPoolExecutor',
 'concurrent.futures.as_completed',
 'typing.Collection',
 'typing.Any',
 'ipywidgets.Image']

In [271]:
tree.xpath("//*[@class='input_area']")[0]

'\n'

In [131]:
bs = bsoup(driver.page_source, 'html.parser')

In [128]:
len(str(bs.body.findAll(['h1', 'h2', 'h3', 'p'])))

11599

In [134]:
bs.findAll( = " views")

[' views']

In [15]:
tree = html.fromstring(driver.page_source)

In [16]:
text = tree.cssselect('strong')+tree.cssselect('p')+tree.cssselect('h1')+tree.cssselect('h2')+tree.cssselect('h3')
len(str([each.text for each in text]))

10059

In [26]:
driver.get("https://kaggle.com/tarunkr/covid-19-case-study-analysis-viz-comparisons")

In [43]:
driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="rendered-kernel-content"]'))

In [71]:
tree = html.fromstring(driver.page_source)
#//*[@id="kernel-header-wrapper"]/div[1]/span[1]/span[2]/span/span[2]/span

In [74]:
tree.xpath('//*[@id="KernelViewerContext_KernelSubtitle-sc-rltxca jxVKtW"]/span[2]/span')[0].text

IndexError: list index out of range

In [75]:
tree.xpath('//*[@id="kernel-header-wrapper"]/div[1]/span[1]/span[2]/span/span[2]/span')[0].text

'11,204'

# requests/ lxml rewrite

In [131]:
page = requests.get("https://www.kaggleusercontent.com/kf/30719380/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..8bzKldvxCkXAaypbyImZTg.s9-xLIUZe8uV6U3re40aWvYORBBUfVkjJWhGuD6SiG1g7hORbv7F1d9_UGjYFh26tgUlb5veTSY0E2zuRpsVf_Jo6kPGq9T20ezNJ7s6phV8z3s0e5kloj7cW1ms6ElD3YHgHm9bEgZqhgRU5OG7tpS-e6SajA6Um2RdcPTZr7yPujY5KsK_dYQg0f15xlnO1GXoo3f6rut9A47t7Neox2NaArx1SnDRcKM7mm_Gv8z2huQ5PkPCFEE5NoA8rB_AkhqZKeF9x8ClSIDl02xEgu6gbp_nGAF9mIiop4V4DkmCLA9hyVuzFITEYxEHFtC3zzptP0U5h3sdEMY20AVhPsR64rdsS95-7ut2Jrw_N0qctWs1cZqjQ_FhipUNx5j2SW9sSXVRS2wjCKZwmVut4wPiXgYi0CgVEG1gybEqkYdqghkdfFSXMh_07OquPyhaFqGwVjCTseZynIQxSZwjnNH8xMRgcryDNrAycKwREvdF69jxv4UtqRSbgquXWdWhIFCXjEaYr6TTXFIJYiNt8_QYgwJwPraIL71aWC0J0HtXvquaxDOrSPzoxeU3HkwPx8UuDk1Fwf-ExFC9XlNWZ8gzCapKjS_n1jY-ph99iy0MS3OTryZQN2Hv6n2hc--ajUP67H2J0VAG-hP4jMnWz_uMu48g4tom3xj5c3KYwcqOkFAT17ZCDrLdFKkwq7_g.2VCKvbXlYIwnWYPyLzv8Cg/__results__.html")

In [132]:
tree = html.fromstring(page.content)

In [135]:
tree.xpath('//*[@class="_kg_hide-input-true"]')

[]

In [127]:
tree = html.fromstring(driver.page_source)

In [130]:
html.tostring(tree.xpath('//*[self::iframe]')[0])

b'<iframe id="rendered-kernel-content" src="https://www.kaggleusercontent.com/kf/30719380/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..LJKthxuWaLqdSnSerftX3Q.jUAwCpIn5a4AcPc6b4Ds7G0WKFnvT_FDupjOuf9kr15EsP1wxnBEspVLmCK606tMWDVNTfciOmiPSN3hs4njXZdqdroSItBNupQunj9NBIUw6T6MLIT6EzZ3tmO9biEiPqlvZhHf6enTdl3iK_KJ67Qt1uUlwT8r7Wh6RU9PkZYAmBAMzaIQdO41nfomO0MrlP6BDrpr-JGT3Z-cQN7CL_v3sU8LMys0ZdWEhUiU-3G5XiU2x6tBlE3wQEIdupUjRJqiLhH4ZYoCnMuyPqBF91Bzu5DSG29SVvLmIEKeOEy7QUlMr_0C38tHrXZ1Jv-KLxmRTdqfSXiHtaViP1mdwmwHK3DHj_m7XjxAYLukPVLpFxPdwxcWeKzp_3TaeyfkplM79Sc_bzw4JJl8QnXij2i9v6hbF1TqJS8ER4jShS-MC93mR2FHhxXMDBBeGp7sVxEyHRql0OEp0u4-RlWNqXUaHkdTH-6kNkEk9icKIRLmEt9WxQ0LZG0J0y4BhgPNSQEL5i8XCn4zcecqHNWDUk9eP3nDc0WUoUE7cIpWkPIIi9VuVyNdWqUHeYdAxbcui76KG3aIg0Z42yRbsVx_5cBFL1QLjejLBA-qgUOK2mwrNf809Jca9Z8eEjjqb1oKCJSMlDM-rOanLhX1THAAvA27NWgRrLYZtFkEVKwXYebbrWv-lyIn5YADyKY2PdVD.y0yzInQarTfOB-G47hKqDg/__results__.html" scrolling="no" title="Main Kernel Content" class="sc-erNlkL hpVXSS" style="height: 46865px; 

In [75]:
tree.xpath('//*[@id="kernel-header-wrapper"]')

[]

In [76]:
views = []
char_length = []
packages = []
for link in meta_df['notebook_link']:
    print (link)
    driver.get(link)
    tree = html.fromstring(driver.page_source)
    
    #get views before switching to iframe
    views_=tree.xpath('//*[@id="kernel-header-wrapper"]/div[1]/span[1]/span[2]/span/span[2]/span')[0].text
    views.append(views_)
    print(f'{views_} views')
    
    #switch to iframe context
    driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="rendered-kernel-content"]'))
    tree = html.fromstring(driver.page_source)
    
    #unhide codeblocks
    codeblocks = driver.find_elements_by_class_name('_kg_hide-input-true')
    for to_unhide in codeblocks:
        driver.execute_script(f"arguments[0].className = '_kg_hide-input-false';", to_unhide)
    
    #scrape imports
    code = [each.text for each in driver.find_elements_by_class_name("input_area") if 'import' in each.text]
    modules = []
    for block in code:
        for each in block.split('\n'):
            tokens = each.split(' ')
            if tokens[0] == 'import':
                for package in re.sub(' ', '', each.split(' as ')[0].split('import')[1]).split(','):
                    modules.append(package)
            elif tokens[0] == 'from':
                submodules = each.split('import')[1]
                for sub in re.sub(' ', '', submodules).split(','):
                    modules.append(tokens[1]+'.'+sub)
    packages_= list(dict.fromkeys(modules))
    packages.append(packages_)
    print(f'{len(packages_)} module(s)')
    text = tree.cssselect('strong')+tree.cssselect('p')+tree.cssselect('h1')+tree.cssselect('h2')+tree.cssselect('h3')
    char_length_ = len(str([each.text for each in text]))
    char_length.append(char_length_)
    print(f'{char_length_} characters')

https://kaggle.com/tarunkr/covid-19-case-study-analysis-viz-comparisons
11,207 views
22 module(s)
5537 characters
https://kaggle.com/dgunning/browsing-research-papers-with-a-bm25-search-engine
16,897 views
7 module(s)
3005 characters
https://kaggle.com/mobassir/mining-covid-19-scientific-papers
5,154 views
66 module(s)
7784 characters
https://kaggle.com/dirktheeng/anserini-bert-squad-for-context-corpus-search
1,139 views
14 module(s)
7925 characters
https://kaggle.com/duttadebadri/covid-19-analysing-growth-factor-inflection
2,173 views
13 module(s)
4069 characters
https://kaggle.com/mlconsult/transmission-incubation-and-environment-2-0
71 views
6 module(s)
2942 characters
https://kaggle.com/phyothuhtet/document-clustering-kmeans-som
357 views
17 module(s)
6570 characters
https://kaggle.com/davidmezzetti/cord-19-analysis-with-sentence-embeddings
5,794 views
16 module(s)
6998 characters
https://kaggle.com/gatunnopvp/covid-19-in-brazil-predicting-cases-and-deaths
835 views
19 module(s)
23

540 views
6 module(s)
2755 characters
https://kaggle.com/robbieedwards/covid-19-lda-notebook
42 views
15 module(s)
302 characters
https://kaggle.com/cedna198/visualization-of-virus-origin-genetic-evolution
93 views
32 module(s)
2341 characters
https://kaggle.com/albertoferrari/most-mentioned-antivirals
1,141 views
10 module(s)
2276 characters
https://kaggle.com/panosc/epidemiological-curves-using-ecdc-data
1,625 views
0 module(s)
2118 characters
https://kaggle.com/docxian/cord-19-metadata-evaluation
2,026 views
12 module(s)
257 characters
https://kaggle.com/luisblanche/cord-19-match-articles-to-tasks-w-doc2vec
2,144 views
16 module(s)
4042 characters
https://kaggle.com/dangelov/covid-19-topic-modeling-and-search-with-top2vec
72 views
5 module(s)
1562 characters
https://kaggle.com/arturkiulian/coronawhy-org-task-risk-factors
63 views
1 module(s)
6378 characters
https://kaggle.com/thebooort/epidemiology-math-models-used-in-research
144 views
9 module(s)
3353 characters
https://kaggle.com

118 views
15 module(s)
2842 characters
https://kaggle.com/arpitrathi/utilizing-biobert-for-k-means-topic-clustering
192 views
18 module(s)
2 characters
https://kaggle.com/dgunning/a-cord19-research-paper-search-engine
227 views
1 module(s)
2094 characters
https://kaggle.com/midnitekoder/covid-19-citation-graph-embedding-using-deepwalk
337 views
12 module(s)
125 characters
https://kaggle.com/otayeby/cord-19-parse-docs-r
134 views
0 module(s)
2 characters
https://kaggle.com/mpwolke/diagnosing-covid-19
633 views
26 module(s)
11790 characters
https://kaggle.com/guymitch2007/covid-19-top-scholarly-journals
696 views
8 module(s)
2120 characters
https://kaggle.com/houssemayed/lda-information-visualization-from-cord-19
364 views
15 module(s)
300 characters
https://kaggle.com/jonathancarvalho/keywords-on-the-subject-ethical-and-social
784 views
14 module(s)
447 characters
https://kaggle.com/alizahidraja/covid-19-find-the-right-research-paper-with-tags
373 views
8 module(s)
3556 characters
https

TimeoutException: Message: timeout: Timed out receiving message from renderer: -387.502
  (Session info: chrome=80.0.3987.149)


203

In [98]:
code.split('\n')[9].split('import')

['', ' json, requests']

In [51]:
driver.switch_to.default_content()

In [19]:
links = [each for each in meta_df['notebook_link']]

In [3]:
driver = Chrome(webdriver)

In [126]:
driver.get("https://kaggle.com/tarunpaparaju/covid-19-dataset-gaining-actionable-insights")

In [10]:
driver.get("https://kaggle.com/dirktheeng/anserini-bert-squad-for-context-corpus-search")

In [12]:
driver.find_element_by_tag_name('iframe').get_attribute('src')

'https://www.kaggleusercontent.com/kf/30719380/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..7NF9JzxoE7s_r21dimn98w.Gt221Q7ef-MAFQMyIbx26MDv6G_lYzvJQgPTWF-YbNcqZQOMo3hJ6x_C6Y96DdrOSJkccIZ09mj8AFklRwAjRLj476eo3n_g-ceBJjgxPPB4ed1uby4ClV-fPz9qhPYbR5S4upIbzbNm2wngW8RFB8gjbry0e3AOLMBb3_YkZ22eTF-n8g8wDqt23PeUhxXoEyZz6ebsP39h901l-Af8NBtm4ZqJepUSNiYyiuLv7eAVZQxHCtY7tJoH_L77Yj1DAeBlV6bcfr4J6fMCNYH0mmTI9M68Cx_9DEGW1P4ytKG9eg1u1tv10to0-RhJwlilG2enpj0Jt2uVHO01jjBRrfPPSYSQG29I6gJPt9u-gNjseCigb9nAKCODcWMLNpDykttpn4839TzaxDHiDRqwjlUa0ma_Duta7AjhF3i-WzLc6WJw-7xAnThj9gQwRxzlRDJ9hOXCBQQQuhLL-H3fJ381bRuMHrZI9kFSnFiiP736m7JfKAkwF8w0fopka44iYUYyrsHwny2lxYYgBmsSRS38kmC9NCmyHv934EpQbtu-dQnw9jovXETEytFcviyNWr9VFDZlwiNBml3cNPWq2y80ZwGR9w_g5pSHQPpSSC6vlOuNeXSo-7IdpK6TdvOuMWp3nS_ZELlp5fdoKIFoRc6dksWOiYwYtYAXnWBdPoZzXLwnFtXJKYa3RD4JXtek.f2kzlMErsFSX-D5WbOqAwg/__results__.html'

In [73]:
driver.switch_to.frame(driver.find_element_by_xpath('//*[@id="rendered-kernel-content"]'))
codeblocks = driver.find_elements_by_class_name('_kg_hide-input-true')
for to_unhide in codeblocks:
    driver.execute_script(f"arguments[0].className = '_kg_hide-input-false';", to_unhide)


In [74]:
codeblocks = driver.find_elements_by_class_name('_kg_hide-input-true')
for to_unhide in codeblocks:
    driver.execute_script(f"arguments[0].className = '_kg_hide-input-false';", to_unhide)


In [65]:
driver.execute_script(f"return document.getElementById('rendered-kernel-content')")

In [33]:
list(dict.fromkeys(['a', 'a', 'a', 'b']))

['a', 'b']

In [31]:
driver.find_element_by_xpath('//*[@id="notebook-container"]').text

"Presenting the CORD19 Research Engine\nAs the COVID-19 pandemic accelerates at frightening pace throughout the world's population, there is an urgent need for solutions or ideas on how to stop it. There are many thousands of research papers on diseases - specific to conronavirus or not; on epidemics and how to fight them; on information gathering and sharing. We hope that we can find among these research papers specific ideas, research or data that give use the ammunition to stop SARS-COV-2 as quickly as we can.\nThis notebook presents a search engine that is built on top of the CORD-19 Research Dataset. It uses both the metadata.csv file and the research paper JSON files for its indexing source, and provides powerful yet simple tool ways to show results to users based on their queries. It is intended for any researcher - be it a nurse at a hospital in Bergamo, or a city planner in Brooklyn or casual data hunters looking for any information that can provide any help.\nFinally, the sea

In [40]:
codeblocks = driver.find_elements_by_class_name('_kg_hide-input-true')
for to_unhide in codeblocks:
    driver.execute_script(f"arguments[0].className = '_kg_hide-input-false';", to_unhide)
code = [each.text for each in driver.find_elements_by_class_name("input_area") if 'import' in each.text]
modules = []
for block in code:
    for each in block.split('\n'):
        tokens = each.split(' ')
        if tokens[0] == 'import':
            for package in re.sub(' ', '', each.split(' as ')[0].split('import')[1]).split(','):
                modules.append(package)
        elif tokens[0] == 'from':
            submodules = each.split('import')[1]
            for sub in re.sub(' ', '', submodules).split(','):
                modules.append(tokens[1]+'.'+sub)


In [46]:
code = [each.text for each in driver.find_elements_by_class_name("input_area") if 'import' in each.text]

In [56]:
driver.find_elements_by_class_name("input_area")[0].find_element_by_tag_name('span').text

''

In [4]:
driver.get("https://kaggle.com/tarunpaparaju/covid-19-dataset-gaining-actionable-insights")

KeyboardInterrupt: 

# Analysing modules

In [139]:
feat = pd.read_csv('notebook_features.csv')

In [159]:
cleaned = feat[feat.packages != '[]'][feat.packages!="['']"]

C:\Users\alexd\Anaconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [165]:
unique = []
for modules in cleaned['packages']:
    for each in eval(modules):
        if each not in unique:
            unique.append(each)

In [187]:
all_modules = {each:0 for each in unique}

In [188]:
for modules in cleaned['packages']:
    for each in eval(modules):
        all_modules[each] +=1

In [175]:
list(all_modules.values())

TypeError: 'dict_values' object does not support indexing

In [192]:
all_modules = {k: v for k, v in sorted(all_modules.items(), key = lambda item: item[1],reverse = True)}

In [193]:
all_modules

{'pandas': 415,
 'numpy': 362,
 'os': 281,
 'json': 227,
 'matplotlib.pyplot': 207,
 're': 134,
 'nltk.corpus.stopwords': 98,
 'seaborn': 97,
 'nltk': 86,
 'spacy': 84,
 'glob': 79,
 'collections.Counter': 65,
 'wordcloud.WordCloud': 62,
 'tqdm.tqdm': 54,
 'sklearn.feature_extraction.text.CountVectorizer': 52,
 'warnings': 51,
 'plotly.express': 49,
 'sklearn.feature_extraction.text.TfidfVectorizer': 42,
 'sys': 42,
 'nltk.tokenize.word_tokenize': 42,
 'gensim': 40,
 'wordcloud.STOPWORDS': 39,
 'nltk.stem.WordNetLemmatizer': 37,
 'tqdm.notebook.tqdm': 37,
 'IPython.core.display.HTML': 34,
 'IPython.display.display': 34,
 'string': 34,
 'random': 34,
 'sklearn.cluster.KMeans': 31,
 'IPython.core.display.display': 31,
 'time': 31,
 'pprint.pprint': 26,
 'pathlib.Path': 26,
 'sklearn.decomposition.PCA': 25,
 'pickle': 25,
 'sklearn.manifold.TSNE': 24,
 'ipywidgets.interact': 23,
 'copy.deepcopy': 23,
 'gensim.models.Word2Vec': 23,
 'requests': 22,
 'sklearn.decomposition.LatentDirichletAl

In [213]:
output_file("modules.html")
p = figure(x_range= list(all_modules.keys()), plot_height = 2000, plot_width = 10000, title = "Modules Used for Covid-19 Data Analysis", toolbar_location=None, tools="")
p.vbar(x=list(all_modules.keys()), top=list(all_modules.values()), width=0.5)
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = "vertical"
show(p)